In [ ]:
!curl -L -O https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length in characters: ", len(text))

In [ ]:
print(text[:1000])

In [ ]:
# Getting the unique characters in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
# stoi -> string to integer, itos -> integer to string
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # take a string and output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # take a list of integers and output a string

print(encode("hi there"))
print(decode(encode("hi there")))

In [ ]:
# Encoding the whole dataset and storing it into a Tensor
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

In [ ]:
# Splitting dataset into training (90%) and testing (10%)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# Trial with block_size = 8
block_size = 8
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target is {target}")

In [ ]:
# same seed to generate same corresponding random numbers
torch.manual_seed(1337)
batch_size = 4 # how many sequences to be processed in parallel 
block_size = 8 

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Context
    x = torch.stack([data[i:i+block_size] for i in ix])
    # Target
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)

print('------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()}, the target: {target} ")

In [ ]:
# Input to the transformer
print(xb)

In [ ]:
# Logits are outputs of a neural netwrok before activation function is applied
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
device = 'cuda'

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size, device=device)
    
    def forward(self, idx, targets= None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # focus on last time step (T)
            logits = logits[:, -1, :]
            # apply softmax to get probability distribution
            probs = F.softmax(logits, dim=-1)
            # sample 
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            #append sampled idx to running sequence
            idx = torch.cat([idx, idx_next], dim=1) # (B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
m = m.to(device)
xb = xb.to(device)
yb = yb.to(device)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(m.generate(idx, 100)[0].tolist()))

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
# Creating a PyTorch Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)


In [ ]:
batch_size = 32
for steps in range(10000):
    # sample batch of data
    xb, yb = get_batch('train')
    
    xb = xb.to(device)
    yb = yb.to(device)
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

In [ ]:
idx = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(m.generate(idx, 500)[0].tolist()))

Self-Attention

In [ ]:
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [ ]:
x[0]

In [ ]:
xbow[0]

In [ ]:
torch.tril(torch.ones(3,3))

In [ ]:
# Version 2
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow, xbow2)

In [ ]:
xbow[0], xbow2[0]

In [ ]:
# Version 3: use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
# Version 4: Self-Attention
torch.manual_seed(1337)
B,T,C = 4,8,332
x = torch.randn(B,T,C)

# Try it with a single head first 
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)
q = query(x)
v = value(x)
wei = q @ k.transpose(-2,-1)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ v
out.shape

In [28]:
class BatchNorm1d:
    
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma=torch.ones(dim)
        self.beta = torch.zeros(dim)
        
    def __call__(self,x):
        # calculate forward pass
        # 1 to normalize the rows
        xmean = x.mean(1, keepdim=True) # batch mean
        xvar = x.var(1, keepdim=True) # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32,100)
x = module(x)

torch.Size([32, 100])